Plotting example paths as alpha varies

In [1]:
import osmnx
import networkx
import pandas
import numpy
import time
import itertools
import pickle
import importlib
import multiprocessing
import sys
import folium
import folium.plugins
import folium.features
from folium import IFrame
from selenium import webdriver
import os

In [2]:
tiles="StamenTerrain"
Manhattan_center=(40.766045,-73.977784)

In [4]:
dfdict = pandas.read_pickle('FINAL_Manhattan_ALL/data.p')
dfdict[0.01].head()

multiplicity  trip_time  trip_APM  trip_length  \
origin_node destination_node                                                   
42421728    42427381                     1        NaN       NaN          NaN   
            42428678                     1        NaN       NaN          NaN   
            42429876                     1        NaN       NaN          NaN   
            42431490                     1        NaN       NaN          NaN   
            42434072                     1        NaN       NaN          NaN   

                              trip_APVM  trip_num_trips  
origin_node destination_node                             
42421728    42427381                NaN             NaN  
            42428678                NaN             NaN  
            42429876                NaN             NaN  
            42431490                NaN             NaN  
            42434072                NaN             NaN

In [5]:
dataDict = {alpha: dataFrame[~(numpy.isnan(dataFrame['trip_time']))] for (alpha,dataFrame) in dfdict.items()}

dataDict[0.01].tail()
  

multiplicity   trip_time  trip_APM  trip_length  \
origin_node destination_node                                                    
2324482224  42440743                     1  177.686331  0.013212      914.547   
            42455887                     1  235.691149  0.013475     1304.880   
2821304136  42428420                     1  392.924407  0.013155     1593.184   
            42431617                     1  165.401229  0.014664      676.248   
2821304137  42442247                     1  780.788280  0.019622     3059.875   

                              trip_APVM  trip_num_trips  
origin_node destination_node                             
2324482224  42440743           0.000006     2598.953125  
            42455887           0.000002     8543.371795  
2821304136  42428420           0.000005     3112.710937  
            42431617           0.000008     2605.104167  
2821304137  42442247           0.000008     2525.993490

In [6]:
trip = (2821304137, 42442247)

In [7]:
for (alpha,df) in dataDict.items():
    print("(alpha: " + str(alpha) + ", Accidents: " + str(df.loc[trip,'trip_APM'] * df.loc[trip,'trip_length'] * 24)+")")

(alpha: 0.01, Accidents: 1441.0)
(alpha: 0.1, Accidents: 1441.0)
(alpha: 0.2, Accidents: 1441.0)
(alpha: 0.3, Accidents: 1441.0)
(alpha: 0.4, Accidents: 1441.0)
(alpha: 0.5, Accidents: 1441.0)
(alpha: 0.6, Accidents: 1441.0)
(alpha: 0.7, Accidents: 1015.0)
(alpha: 0.8, Accidents: 963.0)
(alpha: 0.9, Accidents: 963.0)
(alpha: 0.99, Accidents: 963.0)


In [8]:
def getPoint(node,keyword):
    temp=G.node[node]
    (lat,lng,count)=(temp["y"],temp["x"],temp[keyword])
    if count>0:
        return (lat,lng,count)
    else:
        return None

In [9]:
config=importlib.import_module('MAIN_config_hour_of_day')

windower=config.windower(0)

Manhattan whole day


In [10]:
osmnx.config(log_file=True, log_console=True, use_cache=True)
G_raw = osmnx.graph_from_place('Manhattan Island, New York, USA', network_type='drive')
G=networkx.DiGraph(G_raw.copy())

In [11]:
def set_cost(alpha):
    for begin,end in G.edges:
        G[begin][end]["cost"]=(1-alpha)*G[begin][end]["time"]+alpha*G[begin][end]["accidents"]

In [12]:
	print("INITIALIZING ACCIDENT DATA")
	accidents_raw=config.accidents(windower.timewindower)
	accidents=accidents_raw.copy()
	accidents.set_index("node",drop=True,inplace=True)
	accidents=accidents.groupby(level="node").size()
	networkx.set_edge_attributes(G,0,"accidents") 
	#set default value of zero accidents
	for node,count in accidents.iteritems():
		try: 
			for edge in G.in_edges(node):
				G.edges[edge]["accidents"]=count/windower.window_length
		except Exception: #fails if node is not in graph 
			pass

INITIALIZING ACCIDENT DATA


In [13]:
networkx.set_node_attributes(G,0,"accident_count")
for node,count in accidents.iteritems():
    try:
        G.node[node]["accident_count"]=count
    except Exception:
        pass

In [14]:
print("INITIALIZING TRAVELTIME DATA")
traveltimes_raw=config.traveltimes(windower.timewindower)
traveltimes=traveltimes_raw.copy()
traveltimes['travel_time'] = traveltimes['travel_time'].multiply(traveltimes['num_trips'])
traveltimes.set_index(["begin_node","end_node"],drop=True,inplace=True)
gb = traveltimes.groupby(level = ['begin_node','end_node'])
traveltimes_means = pandas.DataFrame(gb['travel_time'].sum())
traveltimes_means['num_trips'] = gb['num_trips'].sum()
traveltimes_means = traveltimes_means['travel_time']/traveltimes_means['num_trips']
#set default value of infinite time. 
networkx.set_edge_attributes(G,numpy.inf,"time") 
for edge,traveltime in traveltimes_means.iteritems():
	try:
		G.edges[edge]["time"]=traveltime
	except Exception: #fails if (begin,end) is not in graph
		pass

INITIALIZING TRAVELTIME DATA


In [15]:
	print("INITIALIZING TRIP DATA")
	trips_raw=config.trips(windower.timewindower)
	trips=trips_raw.copy()

	# randomize order of trips
	# if config.N_trips is None, use all trips
	numpy.random.seed(0)
	trips=trips.sample(frac=1)
	
	trips=trips[["origin_node","destination_node"]]
	trips.set_index(["origin_node","destination_node"],drop=True,inplace=True)

INITIALIZING TRIP DATA


In [16]:
def path_to_edges(path):
    return [(path[i],path[i+1],0) for i in range(0,len(path)-1)]

def plot_path(alpha,trip,map=None,color=None):
    (O,D)=trip
    set_cost(alpha)
    cheapest_path=networkx.shortest_path(G,source=O,target=D,weight="cost")
    #print(cheapest_path)
    test = G_raw.edge_subgraph(path_to_edges(cheapest_path))
    myplot=osmnx.plot.plot_graph_folium(test,graph_map=map,edge_width=10,
                                    edge_color=color)
    return myplot

In [17]:
lat1 = G.nodes[trip[0]]['y']
lon1 = G.nodes[trip[0]]['x']

lat2 = G.nodes[trip[-1]]['y']
lon2 = G.nodes[trip[-1]]['x']

print(G.nodes[trip[0]])
print(G.nodes[trip[-1]])
print(lat1)
print(lon1)

{'y': 40.7130447, 'x': -74.0072254, 'osmid': 2821304137, 'highway': 'traffic_signals', 'accident_count': 31}
{'y': 40.7223037, 'x': -73.9971275, 'osmid': 42442247, 'highway': 'traffic_signals', 'accident_count': 40}
40.7130447
-74.0072254


In [18]:
accident_heat_data=[getPoint(node,"accident_count") for node in G.node]
accident_heat_data=[datum for datum in accident_heat_data if datum is not None]

In [22]:
myplot=folium.Map(Manhattan_center,
                  zoom_start=12,max_zoom=15,
                  tiles=tiles)

folium.plugins.HeatMap(accident_heat_data,radius=25,blur=70).add_to(myplot)
colorList = ["magenta","orange","yellow","brown","green","pink","purple","blue","teal","red"]
for i,alpha in enumerate(config.alphaList):
    myplot=plot_path(alpha,trip,map=myplot,color=colorList[i])
    
folium.features.Marker((lat1,lon1),icon=folium.Icon(icon=None,color='green')).add_to(myplot)
folium.features.Marker((lat2,lon2),icon=folium.Icon(icon=None,color='red')).add_to(myplot)

legend_html = '''
     <div style="position: fixed; 
     top: 10px; left: 10px; width: 270px; height: 100px; background-color: white;
     border:2px solid grey; z-index:9999; font-family: Arial, Helvetica, sans-serif; font-size:15px;
     "><div class="container-fluid" style="width: 270px; height: 100px"><div class="row"> <div class="col-xs-4">Path $p$</div><div class="col-xs-4">$\\mathbf{A}(p)$</div><div class = "col-xs-4"></div></div>
      <div class="row"><div class="col-xs-4"> $p^*_{.01}(\color{green}{O},\color{red}{D})$ </div>  <div class="col-xs-4">1441</div><div class="col-xs-4">  <div style="background-color: purple; float: right; margin-right: 10px; margin-top: 5px; width: 70px; height: 10px"
                  ></div></div></div>
     <div class="row"><div class="col-xs-4"> $p^*_{0.7}(\color{green}{O},\color{red}{D})$</div><div class="col-xs-4">1015</div><div class="col-xs-4"> <div style="background-color: blue; float: right; margin-right: 10px; margin-top: 5px; width: 70px; height: 10px"
                  ></div></div></div>
     <div class="row"><div class="col-xs-4">$p^*_{.99}(\color{green}{O},\color{red}{D})$</div><div class="col-xs-4">963</div><div class="col-xs-4"><div style="background-color: red; float: right; margin-right: 10px; margin-top: 5px; width: 70px; height: 10px"
                  ></div></div></div>
       </div>           
      </div>
     '''
mjax = ''' <script type="text/javascript" src="mathjax\
/MathJax.js"></script>'''

mjax_config = '''<script type="text/x-mathjax-config">
  MathJax.Hub.Config({
    extensions: ["tex2jax.js"],
    jax: ["input/TeX","output/HTML-CSS"],
    tex2jax: {inlineMath: [["$","$"],["\\(","\\)"]]}
  });
</script>'''
myplot.get_root().header.add_child(folium.Element(mjax_config))
myplot.get_root().header.add_child(folium.Element(mjax))
myplot.get_root().html.add_child(folium.Element(legend_html))

In [23]:
#WARNING#
#DISPLAY HERE WILL NOT NECESSARILY REFLECT THE CROPPED DISPLAY, SEE NEXT CELL#

myplot

In [24]:
fname="pathEx"
myplot.save(fname+".html")
browser=webdriver.Chrome()
browser.get("file://" + os.getcwd()+"/"+fname+".html")
browser.set_window_size(500,750)
#Give time for mathjax to work its magic
time.sleep(5)
browser.save_screenshot(fname+".png")
#browser.quit()

True

In [392]:
myplot2=folium.Map(Manhattan_center,
                  zoom_start=12,max_zoom=15,
                  tiles=tiles)

folium.plugins.HeatMap(accident_heat_data,radius=25,blur=70).add_to(myplot2)
colorList = ["magenta","orange","yellow","brown","green","pink","purple","blue","teal","red"]
for i,alpha in enumerate(config.alphaList):
    myplot=plot_path(alpha,(trip[1],trip[0]),map=myplot2,color=colorList[i])
    
folium.features.Marker((lat1,lon1),icon=folium.Icon(icon=None,color='red')).add_to(myplot2)
folium.features.Marker((lat2,lon2),icon=folium.Icon(icon=None,color='green')).add_to(myplot2)

In [393]:
myplot2